<a href="https://colab.research.google.com/github/shivagamings2005/house_price_predict/blob/main/face_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import pywt
from google.colab.patches import cv2_imshow

In [ ]:
def har(img,mode,level):
  img2=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  img2=np.float32(img2)
  img2/=255
  coeffs=pywt.wavedec2(img2,mode,level)
  coeffs_h=list(coeffs)
  coeffs_h[0]*=0
  img2_h=pywt.waverec2(coeffs_h,mode)
  img2_h*=255
  img2_h=np.uint8(img2_h)
  return img2_h

In [ ]:
import os
folders=os.listdir("/content/drive/MyDrive/filtered_images")
folders_path={}
for i in folders:
  folders_path[i]="/content/drive/MyDrive/filtered_images/"+i;
folders_path

{'robert downey jr': '/content/drive/MyDrive/filtered_images/robert downey jr',
 'chris hemsworth': '/content/drive/MyDrive/filtered_images/chris hemsworth',
 'emma myers': '/content/drive/MyDrive/filtered_images/emma myers',
 'pepper potts': '/content/drive/MyDrive/filtered_images/pepper potts',
 'Benedict Cumberbatch': '/content/drive/MyDrive/filtered_images/Benedict Cumberbatch'}

In [ ]:
x,y=[],[]
for i,name in enumerate(folders_path):
  for j in os.listdir(folders_path[name]):
    img=cv2.imread(folders_path[name]+"/"+j)
    scalled_raw=cv2.resize(img,(32,32))
    img_har=har(img,'db1',5)
    scalled_har=cv2.resize(img_har,(32,32))
    combined=np.vstack((scalled_raw.reshape(32*32*3,1),scalled_har.reshape(32*32,1)))
    x.append(combined)
    y.append(i)

In [ ]:
x=np.array(x).reshape(len(x),4096).astype(float)
x.shape

(197, 4096)

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [ ]:
xtr,xte,ytr,yte=train_test_split(x,y,test_size=2,random_state=0)
pipe=Pipeline([('scaler',StandardScaler()),('svc',SVC(kernel='linear',C=1))])
pipe.fit(xtr,ytr)
pipe.score(xte,yte)

1.0

In [ ]:
print(classification_report(yte,pipe.predict(xte)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       0.50      1.00      0.67         1
           2       1.00      1.00      1.00         1
           4       0.00      0.00      0.00         1

    accuracy                           0.75         4
   macro avg       0.62      0.75      0.67         4
weighted avg       0.62      0.75      0.67         4



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

In [ ]:
model={
    'svm':{
        'model':svm.SVC(gamma='auto',probability=True),
        'param':{
            'svc__C':[1,10,100],
            'svc__kernel':['linear']
        }
    },
    'random_forest':{
        'model':RandomForestClassifier(),
        'param':{
            'randomforestclassifier__n_estimators':[0.1,1,5,10]
        }
    },
    'logistic_regression':{
        'model':LogisticRegression(solver='liblinear',multi_class='auto'),
        'param':{
            'logisticregression__C':[0.1,1,5,10]
        }
    }
}

In [ ]:
scores=[]
est={}
for model_name,param in model.items():
  pipe=make_pipeline(StandardScaler(),param['model'])
  gs=GridSearchCV(pipe,param['param'],cv=5,return_train_score=False)
  gs.fit(xtr,ytr)
  scores.append({
      'model':model_name,
      'best_score':gs.best_score_,
      'best_params':gs.best_params_
  })
  est[model_name]=gs.best_estimator_

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
5 fits failed out of a total of 20.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py", line 340, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages

In [ ]:
import pandas as pd
df=pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.714023,"{'svc__C': 1, 'svc__kernel': 'linear'}"
1,random_forest,0.510575,{'randomforestclassifier__n_estimators': 10}
2,logistic_regression,0.702069,{'logisticregression__C': 0.1}


In [ ]:
est['svm'].score(xte,yte)


1.0

In [ ]:
model=est['svm']

In [ ]:
import joblib
joblib.dump(model,'model.pkl')

['model.pkl']

In [ ]:
name={j:i for i,j in enumerate(folders_path)}
name

{'robert downey jr': 0,
 'chris hemsworth': 1,
 'emma myers': 2,
 'pepper potts': 3,
 'Benedict Cumberbatch': 4}

In [ ]:
import json
with open('name_id.json','w') as f:
  json.dump(name,f)